In [ ]:
import streamlit as st
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
# import selenium
# from selenium import webdriver
# from bs4 import BeautifulSoup


In [28]:
# driver = webdriver.Chrome(executable_path= r"C:\Users\shiv8\Downloads\chromedriver.exe")
# url = "http://www.pro-football-reference.com/years/2019/rushing.htm"
# driver.get(url)
# soup = BeautifulSoup(driver.page_source,'lxml')
# soup = list(soup)




In [3]:
st.title('NFL FOOTBALL Stats (Rushing) Explorer')
st.markdown("""
This app performs the simple web scrapping of NFL FOOTBALL  player Stats data (focusing on Rushing)!
* **Python libraries:** base64,pandas, streamlit, numpy,matplotlib, seaborn.
* **Data Source:** [pro-football-reference.com](https://www.pro-football-reference.com/).


""")

st.sidebar.header('User Input Features')
selected_year = st.sidebar.selectbox('Year', list(reversed(range(1990,2020))))

# web scrapping of NFL player stats
# https://www.pro-football-reference.com/years/2019/rushing.htm

@st.cache

def load_data(year):
    url="https://www.pro-football-reference.com/years/" + str(year) + "/rushing.htm"
    html = pd.read_html(url,header = 1)
    df = html[0]
    raw = df.drop(df[df.Age == 'Age'].index)
    raw = raw.fillna(0)
    playerstats = raw.drop(['Rk'],axis=1)
    return playerstats
playerstats = load_data(selected_year)


# sidebar - Team selection
sorted_unique_team = sorted(playerstats.Tm.unique())
selected_team = st.sidebar.multiselect('Team',sorted_unique_team, sorted_unique_team)

# sidebar - position selection

unique_pos = ['RB', 'QB', 'WR', 'FB', 'TE']
selected_pos = st.sidebar.multiselect('position', unique_pos,unique_pos)

# filtering data

df_selected_team = playerstats[(playerstats.Tm.isin(selected_team)) & (playerstats.Pos.isin(selected_pos))]

st.header('Display player stats of selected Team(s)')
st.write('Data Dimension:' + str(df_selected_team.shape[0]) + ' rows and ' + str(df_selected_team.shape[1]) + ' columns.')
st.dataframe(df_selected_team)

# Download NBA player stats data
# https://discuss.streamlit.io/t/how-to-download-file-in-streamlit/1806
def filedownload(df):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()  # strings <-> bytes conversions
    href = f'<a href="data:file/csv;base64,{b64}" download="playerstats.csv">Download CSV File</a>'
    return href

st.markdown(filedownload(df_selected_team), unsafe_allow_html=True)

# Heatmap
if st.button('Intercorrelation Heatmap'):
    st.header('Intercorrelation Matrix Heatmap')
    df_selected_team.to_csv('output.csv',index=False)
    df = pd.read_csv('output.csv')

    corr = df.corr()
    mask = np.zeros_like(corr)
    mask[np.triu_indices_from(mask)] = True
    with sns.axes_style("white"):
        f, ax = plt.subplots(figsize=(7, 5))
        ax = sns.heatmap(corr, mask=mask, vmax=1, square=True)
    st.pyplot()



InternalHashError: module '__main__' has no attribute '__file__'

While caching the body of `load_data()`, Streamlit encountered an
object of type `builtins.function`, which it does not know how to hash.

**In this specific case, it's very likely you found a Streamlit bug so please
[file a bug report here.]
(https://github.com/streamlit/streamlit/issues/new/choose)**

In the meantime, you can try bypassing this error by registering a custom
hash function via the `hash_funcs` keyword in @st.cache(). For example:

```
@st.cache(hash_funcs={builtins.function: my_hash_func})
def my_func(...):
    ...
```

If you don't know where the object of type `builtins.function` is coming
from, try looking at the hash chain below for an object that you do recognize,
then pass that to `hash_funcs` instead:

```
Object of type builtins.function: <function load_data at 0x00000192AFDF90D0>
Object of type builtins.tuple: ('__main__', 'load_data', <function load_data at 0x00000192AFDF90D0>)
```

Please see 